https://applenob.github.io/tf_6.html

Tensorflow 学习笔记（六） ———— Optimizer

二 31 一月 2017 by Chan

In [46]:
import tensorflow as tf
import numpy as np

In [2]:
# x = tf.placeholder(tf.float32)
x = tf.Variable(tf.truncated_normal([1]), name="x")
goal = tf.pow(x-3,2, name="goal")

In [4]:
with tf.Session() as sess:
    x.initializer.run()
    print(x.eval())
    print(goal.eval())

[0.16881551]
[8.015605]


In [ ]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train_step = optimizer.minimize(goal)

In [100]:
optimizer.minimize?

In [65]:
iter_num = 20

def train_func():
    with tf.Session() as sess:
        x.initializer.run()
        for i in range(iter_num):
            #print('x:', x.eval())
            train_step.run()
            print('i:', i, 'goal:', goal.eval(), 'x:', x.eval())


In [66]:
'''
2 . minimize() = compute_gradients() + apply_gradients()¶
拆分成计算梯度和应用梯度两个步骤。
'''
# compute_gradients 返回的是：A list of (gradient, variable) pairs
gra_and_var = optimizer.compute_gradients(goal)
train_step = optimizer.apply_gradients(gra_and_var)
train_func()

i: 0 goal: [2.1050167] x: [1.5491325]
i: 1 goal: [2.021658] x: [1.5781498]
i: 2 goal: [1.9416003] x: [1.6065868]
i: 3 goal: [1.864713] x: [1.6344551]
i: 4 goal: [1.7908704] x: [1.6617659]
i: 5 goal: [1.7199521] x: [1.6885306]
i: 6 goal: [1.651842] x: [1.71476]
i: 7 goal: [1.5864292] x: [1.7404647]
i: 8 goal: [1.5236065] x: [1.7656554]
i: 9 goal: [1.4632716] x: [1.7903423]
i: 10 goal: [1.4053261] x: [1.8145355]
i: 11 goal: [1.3496752] x: [1.8382448]
i: 12 goal: [1.296228] x: [1.8614799]
i: 13 goal: [1.2448975] x: [1.8842503]
i: 14 goal: [1.1955994] x: [1.9065653]
i: 15 goal: [1.1482537] x: [1.928434]
i: 16 goal: [1.1027828] x: [1.9498653]
i: 17 goal: [1.0591127] x: [1.970868]
i: 18 goal: [1.0171717] x: [1.9914507]
i: 19 goal: [0.97689164] x: [2.0116217]


In [109]:
'''
3. 进一步
clip_by_global_norm:修正梯度值
'''
gradients, variables = zip(*gra_and_var)
# gradients, variables = gra_and_var[0] #导致下面报错: TypeError: t_list should be a sequence
gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
train_step = optimizer.apply_gradients(zip(gradients, variables))
train_func()

i: 0 goal: [7.270593] x: [0.30359626]
i: 1 goal: [7.203339] x: [0.31609625]
i: 2 goal: [7.136399] x: [0.32859623]
i: 3 goal: [7.0697694] x: [0.34109622]
i: 4 goal: [7.0034533] x: [0.3535962]
i: 5 goal: [6.937449] x: [0.3660962]
i: 6 goal: [6.8717575] x: [0.3785962]
i: 7 goal: [6.8063793] x: [0.39109617]
i: 8 goal: [6.741313] x: [0.40359616]
i: 9 goal: [6.676559] x: [0.41609615]
i: 10 goal: [6.6121182] x: [0.42859614]
i: 11 goal: [6.5479894] x: [0.44109613]
i: 12 goal: [6.484173] x: [0.45359612]
i: 13 goal: [6.4206686] x: [0.4660961]
i: 14 goal: [6.357477] x: [0.4785961]
i: 15 goal: [6.294599] x: [0.49109608]
i: 16 goal: [6.232033] x: [0.50359607]
i: 17 goal: [6.1697783] x: [0.51609606]
i: 18 goal: [6.107838] x: [0.52859604]
i: 19 goal: [6.046209] x: [0.54109603]


In [99]:
'''
exponential_decay 加入学习率衰减：¶
'''
# global_step 记录当前是第几个batch
global_step = tf.Variable(0)
learning_rate = tf.train.exponential_decay(
    3.0, global_step, 3, 0.3, staircase=True)
optimizer2 = tf.train.GradientDescentOptimizer(learning_rate)
gradients, vriables = zip(*optimizer2.compute_gradients(goal))
# gradients, _ = tf.clip_by_norm(gradients, 1.25)
gradients, _ = tf.clip_by_global_norm(gradients, 1.25) #必须有这一步, 否则不收敛
train_step = optimizer2.apply_gradients(zip(gradients, vriables), 
                                       global_step=global_step)
# train_step = optimizer2.apply_gradients(zip(gradients, vriables))

with tf.Session() as sess:
        global_step.initializer.run()
        x.initializer.run()
        for i in range(10):
            #print 'x:', x.eval()
            train_step.run()
            print('i:', i, 'goal:', goal.eval(), 'x:', x.eval(),
                  'global_step:', global_step.eval())

# train_func()

i: 0 goal: [0.6555249] x: [3.809645] global_step: 1
i: 1 goal: [8.645688] x: [0.05964494] global_step: 2
i: 2 goal: [0.6555249] x: [3.809645] global_step: 3
i: 3 goal: [0.09944896] x: [2.6846447] global_step: 4
i: 4 goal: [0.06364736] x: [3.2522843] global_step: 5
i: 5 goal: [0.04073435] x: [2.7981725] global_step: 6
i: 6 goal: [0.00861939] x: [2.9071593] global_step: 7
i: 7 goal: [0.00182386] x: [2.9572933] global_step: 8
i: 8 goal: [0.00038593] x: [2.980355] global_step: 9
i: 9 goal: [0.00027102] x: [2.9835374] global_step: 10


In [91]:
learning_rate

<tf.Tensor 'ExponentialDecay_14:0' shape=() dtype=float32>

In [88]:
tf.clip_by_global_norm?

In [89]:
tf.clip_by_average_norm?

In [87]:
tf.clip_by

In [83]:
optimizer.apply_gradients?

In [79]:
tf.train.exponential_decay?